# Normalization over the alphabet $\{0,1,2\}$

In [27]:
import tgpc

## Working with $E_i$-palindromes and closures

### Involutory antimorphisms $E_i$

In [28]:
print(tgpc.Ei("0"))
print(tgpc.Ei("1"))
print(tgpc.Ei("2"))

('0', '2', '1')
('2', '1', '0')
('1', '0', '2')


### Checking if a ternary string is a pseudopalindrome

In [29]:
help(tgpc.is_eipal)

Help on function is_eipal in module tgpc:

is_eipal(seq, i)
    Checks if a word is an Ei-palindrome.
    
    Args:
        seq (string): The word checked composed 
            of the letters "0", "1" and "2".
        i: Pseudopalindromic type, can be either 0, 1, 2, or 
            "0", "1", "2", standing for E_0, E_1 and E_2.
        
    Returns:
        True if the word is an Ei-palindrome, otherwise False.
    
    Examples:
        >>> is_eipal("012", 1)
        True
        >>> is_eipal("002", 1)
        False



In [30]:
print(tgpc.is_eipal("012", "1"))
print(tgpc.is_eipal("002", "1"))
print(tgpc.is_eipal("01201", "2"))

True
False
True


### ... or an $R$-palindrome

In [31]:
print(tgpc.is_pal("00100"))
print(tgpc.is_pal("001200"))

True
False


### Making $E_i$ and $R$-palindromic closures

In [32]:
print(tgpc.make_eipal_closure("102", 0))
print(tgpc.make_eipal_closure("101", 1))
print(tgpc.make_pal_closure("102"))

102
10121
10201


### Creating a GPS word from $(\Delta, \Theta)$

In [33]:
print(tgpc.make_word012("0012112", "R012R11"))

00122201110222100122100021112000122201110222100122100021112000122


The logging level can be set, so that we see the construction:

In [34]:
tgpc.set_logging("INFO")
print(tgpc.make_word012("0012112", "R012R11"))

w1 = 0
w2 = 00
w3 = 00122
w4 = 001222011
w5 = 00122201110222100
w6 = 00122201110222100122100021112000122
w7 = 00122201110222100122100021112000122201110222100122100021112000122


00122201110222100122100021112000122201110222100122100021112000122


... And setting it back to the default level (`ERROR`), otherwise some functions are quite verbose...

In [35]:
tgpc.set_logging("ERROR")

## Normalization algoritm

The new normalization algoritm is implemented in the `Normalized012` class:

In [36]:
delta = "0102110"
theta = "02R0121"
normalizer = tgpc.Normalizer012()
normalizer.normalize(delta, theta)

('01021102', '02R01201', False)

`False` means that the original directive bi-sequence $(\Delta, \Theta)$ was not normalized.

Thus the normalized form of $(0102110, E_0E_2RE_0E_1E_2E_1)$ is $(01021102, E_0E_2RE_0E_1E_2E_0E_1)$. Again, by setting the logging level to `INFO`, the different actions of the normalizer can be shown:

In [37]:
tgpc.set_logging("INFO")
normalizer.normalize(delta, theta)

Checking for an applicable rule in('0102110', '02R0121')
rule3: ('110', '121') in biseq ('0102110', '02R0121')
Checking for an applicable rule in('01021102', '02R01201')
bi-sequence before changing the letters back: (01021102, 02R01201)


('01021102', '02R01201', False)

In [38]:
delta = "02110"
theta = "02R10"
normalizer.normalize(delta, theta)

Checking for an applicable rule in('01220', '01R20')
prefix rule: (re.compile('00(120R)*11'), '1221', 9)
Checking for an applicable rule in('012220', '021R20')
bi-sequence before changing the letters back: (012220, 021R20)


('021110', '012R10', False)

In the first example, the third factor rule $(ab_1b_2, E_iE_jE_i) \rightarrow (ab_1b_2E_iE_j(b_1), E_iE_jE_kE_i), E_i(b_1)= E_j(b_2)$ was used with $(ab_1b_2, E_iE_jE_i) = (110, E_1E_2E_0)$.

In the second, the ninth prefix rule was used: $(ijk,E_iE_kE_k) \rightarrow (ijkij, E_iE_kE_jE_iE_k)$. Note that the algorithm changed the letters order to be successively $0$, $1$ and $2$ in $\mathrm{u} (\Delta, \Theta)$.

Finally, we can compare the results with a naive Normalization algorithm. By letting the log level set to `INFO`, we see that it proceeds by actually generating the suffixes $w_i$ of pseudostandard generalized word from $(\Delta, \Theta)$ and by searching for all the pseudopalindromic prefixes in the word:

In [39]:
delta = "02110"
theta = "02R10"
naive_normalizer = tgpc.NaiveNormalizer012()
naive_normalizer.normalize(delta, theta)

Prefixes from (delta, theta): ['0', '021', '021120', '0211201102', '021120110201022012210']
Obtained word: 021120110201022012210
0
02
021
021120
0211201102
021120110201022012210


('021110', '012R10', False)

The output of both the `NaiveNormalizer` and the `Normalizer` are the same. However, the NaiveNormalizer needs to construct $\mathrm{u} (\Delta, \Theta)$. It has to search for longest pseudopalindromic suffixes during the construction, and then search for all the pseudopalindromic prefixes of the constructed word.

In [40]:
tgpc.set_logging("ERROR")
delta = "010211002101"
theta = "02R0121R0102"
print(normalizer.normalize(delta, theta))
print(naive_normalizer.normalize(delta, theta))

('010211020210210', '02R01201R012012', False)
('010211020210210', '02R01201R012012', False)


In [43]:
%timeit normalizer.normalize(delta, theta)

1.73 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [44]:
%timeit naive_normalizer.normalize(delta, theta)

573 ms ± 40.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
